In [1]:
from sklearn import svm
from skimage import io
from joblib import dump, load
import numpy as np
import os
from skimage.transform import resize

In [2]:
def compute_gradient(image):
    gx = np.zeros((image.shape[0], image.shape[1]))
    gy = np.zeros((image.shape[0], image.shape[1]))
    image = image.astype(np.float32)
    gx[:, 1:-1] = (image[:, 2:] - image[:, :-2]) / 2
    gy[1:-1, :] = (image[2:, :] - image[:-2, :]) / 2
    
    gx[:, 0] = image[:, 1] - image[:, 0]
    gy[0, :] = image[1, :] - image[0, :]
    
    gx[:, -1] = image[:, -1] - image[:, -2]
    gy[-1, :] = image[-1, :] - image[-2, :]
    return gx, gy

In [3]:
def hog_cell(orientations, magnitudes, n):
    bin_size = int(180 / n)
    hog = np.zeros(n)
    for i in range(orientations.shape[0]):
        for j in range(orientations.shape[1]):
            angle = orientations[i, j]
            magnitude = magnitudes[i, j]
            bin = int(angle / bin_size)
            if bin == n:
                bin = n - 1
            hog[bin] += magnitude
            
    return hog/(magnitudes.shape[0]*magnitudes.shape[1])

In [4]:
def normalize_vector(v):
    epsion = 1e-5
    return v / np.sqrt(np.sum(v ** 2) + epsion ** 2) 

In [5]:
def get_hog_featrue(img):
    gx, gy = compute_gradient(img)
    x, y = gx.shape
    cx , cy = 8, 8
    bx , by = 1, 1
    
    magnitude = np.sqrt(gx**2 + gy**2)
    angels = np.rad2deg(np.arctan2(gy, gx)) % 180
    
    n_cells_x = int(x / cx)
    n_cells_y = int(y / cy)
    n_blocks_x = n_cells_x - bx + 1
    n_blocks_y = n_cells_y - by + 1
    
    cells = np.zeros((n_cells_x, n_cells_y, 9))
    prev_x = 0
    for i in range(n_cells_x):
        prev_y = 0
        for j in range(n_cells_y):
            cells[i, j] = hog_cell(angels[prev_x:prev_x+cx, prev_y:prev_y+cy], magnitude[prev_x:prev_x+cx, prev_y:prev_y+cy], 9)
            prev_y += cy
        prev_x += cx
    
    cells_norm = np.zeros((n_blocks_x, n_blocks_y, 9))
    #normalize the cells
    
    for i in range(n_blocks_x):
        for j in range(n_blocks_y):
            cells_norm[i, j] = normalize_vector(cells[i:i+bx, j:j+by].ravel())
            
    return cells_norm.ravel()

In [6]:
def resize_img(img1):
    #resize photo to 200x200
    img_new = resize(img1, (256, 256), anti_aliasing=False)
    return img_new

In [7]:
def normalize_img(img):
    #resize photo so that only the hand is visible
    # get the most left pixel that is not black
    if img.shape[0] == 0 or img.shape[1] == 0:
        return img
    #check indcies are not out of bounds
    most_left = np.where(img.sum(axis=0) != 0)[0][0]
    most_top = np.where(img.sum(axis=1) != 0)[0][0]
    most_right = np.where(img.sum(axis=0) != 0)[0][-1]
    most_bottom = np.where(img.sum(axis=1) != 0)[0][-1]
    resized_img = img[most_top:most_bottom, most_left:most_right]
    return resized_img

In [8]:
def get_paper_hand_imgs():
    imgs = []
    labels = []
    #open folder with paper hands
    #get all images in folder and append to imgs
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/five/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("paper")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("paper")
    return imgs, labels

def get_rad_hand_imgs():
    imgs = []
    labels = []
    #open folder with rock hands
    #get all images in folder and append to imgs
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/rad/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("rad")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("rad")
    return imgs, labels

def get_peace_hand_imgs():
    imgs = []
    labels = []
    #open folder with scissors hands
    #get all images in folder and append to imgs
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/peace/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("peace")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("peace")
    return imgs, labels

def get_fist_hand_imgs():
    imgs = []
    labels = []
    #open folder with thumbs up hands
    #get all images in folder and append to imgs
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/fist/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("fist")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("fist")
    return imgs, labels

def get_thumbsup_hand_imgs():
    imgs = []
    labels = []
    #open folder with thumbs up hands
    #get all images in folder and append to imgs
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/thumbs/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("thumbs")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("thumbs")
    return imgs, labels

def get_perfecto_imgs():
    imgs = []
    labels = []
    weights = []
    #open folder with thumbs up hands
    #get all images in folder and append to imgs
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/okay/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("perfecto")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("perfecto")
    return imgs, labels

def get_straight_imgs():
    imgs = []
    labels = []
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/straight/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        img_flipped = np.fliplr(img)
        img_flipped = normalize_img(img_flipped)
        resized_flipped = resize_img(img_flipped)
        imgs.append(resized_flipped)
        labels.append("straight")
        img = normalize_img(img)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("straight")
    return imgs, labels

def get_none_imgs():
    imgs = []
    labels = []
    path = 'C:/Users/Fastora/Downloads/HandGestureDetection/data/data/data/train/none/'
    for filename in os.listdir(path):
        img = io.imread(path+filename)
        resized = resize_img(img)
        imgs.append(resized)
        labels.append("none")
    return imgs, labels

In [9]:
def extract_features(imgs_labels, feature_vector=True):
    features = []
    labels = []
    for img_label in imgs_labels:
        #get hog features of image and append to features and label to labels
        fd = get_hog_featrue(img_label[0])
        labels.append(img_label[1])
        features.append(fd)
    return features, labels

In [10]:
paper_imgs , paper_labels = get_paper_hand_imgs()
peace_imgs , peace_labels = get_peace_hand_imgs()
fist_imgs , fist_labels = get_fist_hand_imgs()
rad_imgs , rad_labels = get_rad_hand_imgs()
thumbs_imgs , thumbs_labels = get_thumbsup_hand_imgs()
perfecto_imgs , perfecto_labels = get_perfecto_imgs()
straight_imgs , straight_labels = get_straight_imgs()
none_imgs , none_labels = get_none_imgs()

In [11]:
imgs_labels = []
for i in range(len(paper_imgs)):
    imgs_labels.append((paper_imgs[i], paper_labels[i]))
for i in range(len(fist_imgs)):
    imgs_labels.append((fist_imgs[i], fist_labels[i]))
for i in range(len(peace_imgs)):
    imgs_labels.append((peace_imgs[i], peace_labels[i]))
for i in range(len(rad_imgs)):
    imgs_labels.append((rad_imgs[i], rad_labels[i]))
for i in range(len(thumbs_imgs)):
    imgs_labels.append((thumbs_imgs[i], thumbs_labels[i]))
for i in range(len(perfecto_imgs)):
    imgs_labels.append((perfecto_imgs[i], perfecto_labels[i]))
for i in range(len(straight_imgs)):
    imgs_labels.append((straight_imgs[i], straight_labels[i]))
for i in range(len(none_imgs)):
    imgs_labels.append((none_imgs[i], none_labels[i]))

In [12]:
features , labels = extract_features(imgs_labels)

In [13]:
clf = svm.SVC()
clf.fit(features, labels)

SVC()

In [14]:
dump(clf, 'model.joblib')

['model.joblib']